In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from typing import List

In [2]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_fanstat"
api_name = "author_audience_distribution"

In [3]:
fanstat_data = []
fanstat_columns = ["category", "name", "value", "influencer_id"]

In [4]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": blob.name.split('/')[-1].replace(".json", "").split("_")[-1],
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/author_audience_distribution_0913_0.json, 1726205683009498>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': '0'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-13/author_audience_distribution_0913_list300.json, 1726214662873452>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 'list300'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-17/author_audience_distribution_0917_0.json, 1726565274224590>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': '0'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-18/author_audience_distribution_0918_0.json, 1726633852886434>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': '0'},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-09/author_audience_distribution_241009_0.json, 1728452634420312>,
  'date': datetime.datetime(2024, 10, 9, 0, 0),
  'batch_number': '0'}]

In [5]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1],
} for blob in storage_client.list_blobs("4_data_warehouse",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and len(blob.name.split("/")) == 4]
processed_blobs

[{'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fanstat/2024-09-13/douyin_fanstat_240918_0.parquet, 1726715720882245>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': '0'},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fanstat/2024-09-17/douyin_fanstat_240918_0.parquet, 1726715879609934>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': '0'},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fanstat/2024-09-18/douyin_fanstat_240919_0.parquet, 1726717827483806>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': '0'}]

In [7]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 17):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': '0',
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-09/author_audience_distribution_241009_0.json, 1728452634420312>,
  'date': datetime.datetime(2024, 10, 9, 0, 0)}]
1


In [8]:
mapping = {
    "省份分布": "province",
    "年龄分布": "age",
    "设备品牌分布": "device",
    "性别分布": "gender",
}

In [9]:
def get_author_audience_distribution_data(raw_data):
    result = []
    for data in raw_data:
        author_audience_distribution = data.get("author_audience_distribution")
        profile_id = data["profile_id"]
        if author_audience_distribution:
            distributions = author_audience_distribution.get("distributions")
            if distributions:
                for distribution in distributions:
                    if distribution["type_display"] in mapping:
                        items = distribution["distribution_list"]
                        for item in items:
                            record = {}
                            record["category"] = mapping[distribution["type_display"]]
                            record["name"] = item["distribution_key"]
                            record["value"] = item["distribution_value"]
                            record["influencer_id"] = profile_id
                            result.append(record)
    return result


In [10]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    fanstat_data = get_author_audience_distribution_data(raw_data)
    df = pd.DataFrame(fanstat_data, columns=fanstat_columns)
    df.to_parquet("gs://4_data_warehouse/1_xingtu/douyin_fanstat/" + str(item["date"].date()) + "/douyin_fanstat_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")